In [ ]:
%pip install -r requirements.txt

# Imports

In [ ]:
import os
import json

from typing import List
from dotenv import load_dotenv

from pydantic import BaseModel, Field

In [ ]:
from langchain.chains import SimpleSequentialChain
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableMap

In [ ]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_community.utilities.ticker_symbol import TickerSymbolRetriever

In [ ]:
from langfuse import Langfuse

# Load Config

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Access environment variables
azure_api_key = os.getenv("AZURE_API_KEY")
azure_endpoint = os.getenv("AZURE_ENDPOINT")
deployment_name = os.getenv("DEPLOYMENT_NAME")
langfuse_public = os.getenv("LANGFUSE_PUBLIC_KEY")
langfuse_secret = os.getenv("LANGFUSE_SECRET_KEY")

# LLM Configuration

In [ ]:
# Azure OpenAI - GPT-4o or GPT-4o-mini
llm = AzureChatOpenAI(
    deployment_name="your_deployment_name",
    model_name="gpt-4o",
    temperature=0.3,
    api_key="your_azure_openai_api_key",
    azure_endpoint="https://your_resource_name.openai.azure.com/",
    api_version="2024-05-01-preview",
)

# Output Schema

In [ ]:
# Define the output schema using Pydantic
class SentimentOutput(BaseModel):
    company_name: str
    stock_code: str
    newsdesc: str
    sentiment: str
    people_names: List[str]
    places_names: List[str]
    other_companies_referred: List[str]
    related_industries: List[str]
    market_implications: str
    confidence_score: float = Field(ge=0.0, le=1.0)

# Parser
parser = PydanticOutputParser(pydantic_object=SentimentOutput)

# Initialize Langfuse

In [ ]:
# Initialize Langfuse
langfuse = Langfuse(public_key=langfuse_public, secret_key=langfuse_secret)

# Get Relevant Data

In [ ]:
# Step 1 & 2: Accept company name and retrieve stock ticker
def get_stock_ticker(company_name: str) -> str:
    retriever = TickerSymbolRetriever()
    ticker = retriever.run(company_name)
    return ticker

# Step 3: Fetch Yahoo Finance news
def fetch_news(ticker: str) -> str:
    tool = YahooFinanceNewsTool()
    return tool.run(ticker)

# Setup Chain

In [ ]:
# Step 4: Sentiment analysis prompt
sentiment_prompt = PromptTemplate(
    input_variables=["company", "ticker", "news"],
    template="""
Analyze the sentiment of the news for the given company.

Company: {company}
Ticker: {ticker}
News Summaries:
{news}

Return the result using this JSON format:
{format_instructions}
""",
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [ ]:
# Final sentiment chain
sentiment_chain = sentiment_prompt | llm | parser

# Main Pipeline Function

In [ ]:
def run_sentiment_pipeline(company_name: str):
    trace = langfuse.trace(name="sentiment_analysis_pipeline", input={"company": company_name})

    # Extract ticker
    ticker = get_stock_ticker(company_name)
    trace.span("stock_code_extraction").log_input_output(input=company_name, output=ticker)

    # Fetch news
    news = fetch_news(ticker)
    trace.span("news_fetching").log_input_output(input=ticker, output=news)

    # Run sentiment analysis
    input_data = {
        "company": company_name,
        "ticker": ticker,
        "news": news
    }
    result = sentiment_chain.invoke(input_data)
    trace.span("sentiment_parsing").log_input_output(input=input_data, output=result)

    return result

# Test Case

In [ ]:
company = input("Enter company name: ")
result = run_sentiment_pipeline(company)
print(json.dumps(result.dict(), indent=2))